## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Here we will try to extract more data from the describtions of each job using simple NER, POS & Regex expressions<br>
We will also need to clean the data after we extract it and make sure that there isn't any NANs

**Things we will extract from the dexcribtions:**
- Salaries data
- Mentioned skills
- Requeird languages <sub>*(Natural languages)</sub>
- Requeird years of experience
- Requeird or prefered degree

## <center><strong>Importing the <span style= 'color: #48e0dc'>Packeges</span>

In [9]:
import os
import re
import random
import nltk
import warnings
import matplotlib
import pandas             as pd
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt

from bs4                  import BeautifulSoup
from google.cloud         import bigquery
from wordcloud            import WordCloud
from IPython.display      import set_matplotlib_formats
from collections          import Counter

In [6]:
%matplotlib inline
plt.rcParams['font.family'] = 'Candara'

warnings.filterwarnings('ignore')

credentials_path :str = '../credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

FONT             :int  = 17
COLORS           :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                          'uint16', 'uint32', 'uint64']

## <center><strong>Exploring the <span style= 'color: #48e0dc'>Text</span> data
<sub>*And clean the text data.</sub>

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [7]:
project_id :str = 'data-jobs-analysis-db'
dataset_id :str = 'data_jobs_analysis_db'

client   = bigquery.Client(project= project_id)

linkedin_jobs    = client.query(f'SELECT * FROM {dataset_id}.linkedin_jobs;').to_dataframe()
upwork_profiles  = client.query(f'SELECT * FROM {dataset_id}.upwork_profiles;').to_dataframe()
guru_profiles    = client.query(f'SELECT * FROM {dataset_id}.guru_profiles;').to_dataframe()

In [8]:
linkedin_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   jobs_locations      3455 non-null   object
 1   listing_dates       3349 non-null   object
 2   jobs_titles         3455 non-null   object
 3   companies_names     3455 non-null   object
 4   jobs_links          3455 non-null   object
 5   describtion         3455 non-null   object
 6   location_type       3455 non-null   object
 7   employment_type     3455 non-null   object
 8   industry            3414 non-null   object
 9   reqierd_credential  3455 non-null   object
 10  country             3455 non-null   object
 11  job_title           3455 non-null   object
 12  total_jobs          3455 non-null   Int64 
dtypes: Int64(1), object(12)
memory usage: 354.4+ KB


In [10]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def remove_html_tags(text):
    cleantext = re.sub(CLEANR, '', text)
    return cleantext

linkedin_jobs['describtion'].dropna().apply(remove_html_tags)

0       We are seeking a Data Engineer to join the Arc...
1       Job Title: Entry Level Data AnalystbrbrLocatio...
2       Oakland - California - United States of Americ...
3       San Francisco - California - United States of ...
4       Dice is the leading career destination for tec...
                              ...                        
3450    strongemSmile Identity builds trust./em/strong...
3451    Data Engineerpbr/pNew Jersey Connecticut Chelm...
3452    F_IT EIA Core and Compliancepbr/pFull Timepbr/...
3453    Positions: 1 Full Time Associate, Mid-Senior l...
3454    strongTitle: Cloud technologies/Data Analytics...
Name: describtion, Length: 3455, dtype: object

In [14]:
upwork_profiles['describtion'][5]

'Welcome to my Upwork digital portfolio!  \nFor quick insights and to save you time, I have summarized key points of my Data Professional career as follows:\n✅ 4+ years of Azure experience\n     ✔️ Azure Data warehouse | Azure SQL databases\n     ✔️ Azure Data Factory \n     ✔️ Azure Synapse Analytics\n     ✔️ Microsoft PowerBI \n     ✔️ PowerBI integration with Azure Synapse \n✅ 2+ years of Google Cloud experience\n     ✔️ BigQuery Expert\n     ✔️ Google Data Studio \n✅ 2+ years of AWS databases experience \n     ✔️ PostgreSQL | SQL Server databases migration\n     ✔️ MySQL database optimization\n\nUPWORK INSIGHTS \n\n✅ 5+ Upwork Enterprise/corporate clients\n✅ $100k+ Earnings\n✅ 4000+ Work hours\n✅ 200+ completed jobs\n\nAbout Me:\nI am a Certified Azure Data Engineer with 4+ years of corporate experience working on Azure SQL databases, Azure Data Factory Data Pipelines, Azure Synapse Analytics, Enterprise level Data Warehousing and Linked Services via Azure Synapse Analytics, and da

In [23]:
import nltk

# Assuming you have already defined 'upwork_profiles' and 'describtion'

# Step 1: Tokenize the text
text = upwork_profiles['describtion'][5]
tokens = nltk.word_tokenize(text)

# Step 2: Perform POS tagging
tag = nltk.pos_tag(tokens)

pos_tree = nltk.Tree.fromstring(str(tag))

# Step 2: Display the POS tree
pos_tree.pretty_print()

ValueError: Tree.read(): expected '(' but got '['
            at index 0.
                "[('Welcome..."
                 ^